# Crawling broadly with `Scrapy`

## Outline

* [Crawling broadly with `Scrapy`](#scrapy)
    * [A simple (narrow) spider](#simple)
    * [Link extraction in a (broad) spider](#linkextraction)

    
### Credits
Part of this notebook was written by me ([Jaren Haber](https://www.jarenhaber.com/)), part was borrowed from [the official Scrapy tutorial](https://docs.scrapy.org/en/latest/intro/tutorial.html), part came from [this great book on Scrapy](https://learning.oreilly.com/library/view/learning-scrapy/9781784399788/) (you may have University access [here](https://www.safaribooksonline.com/library/view/temporary-access/)), and part of the parsing stuff yesterday was inspired by [Geoff Bacon's web-scraping workshop](https://github.com/TextXD/introduction-to-web-scraping). The last three are great resources for further exploration and learning!

**__________________________________**

# Introduction to `Scrapy` <a id='scrapy'> </a>

As a _Twisted_ application, Scrapy is event-driven, asynchronous, and is virtually multi-threaded (while using only one thread). While other programs cause _blocks_ when they access files or the web, spawn new processes, or do system operations, Scrapy instead waits until a resource is available, solves the immediate problem, and then calls another task. In short, Scrapy is fast, flexible, and scalable. It offers one of the most user-friendly ways to write crawling programs that can move across heterogeneous swaths of the internet, download stuff, and not break. 

To grasp the intuition behind Scrapy, imagine a bank where tellers (threads) are available to see customers (processes), who need to fill out forms before they're done. Such a situation could be configured in these ways:

- _Blocking_ operation with a _single_ thread: Here there is 1 teller trying to help 5 customers. When customer 1 needs time to fill out a form, then teller 1 is occupied waiting for customer 1--and all the other customers are stuck in line.
- _Blocking_ operation with _multiple_ threads: Now there are still 5 customers, but there are 3 tellers. When customer 1 needs time to fill out a form, then teller 1 is occupied. Customer 2 may have access to teller 2 and customer 3 to teller 3, but then all the tellers are monopolized while people fill out forms, which means customers 4 and 5 are still stuck waiting in line. 
- _Non-blocking_ operation with a _single_ thread: Here again we have 1 teller and 5 customers. When customer 1 needs time to fill out a form, they stand aside so the single teller can help customer 2. When customer 1 is finished, they wait until customer 2 is done or has something to do, then customer 1 is called back to continue being helped. If customers 1 and 2 both have forms to complete, they can do that on the side and the single teller can see customer 3, and so on. 

You can see this last situation is way more efficient than the previous two. This is the Scrapy default; when _multiple_ threads are available for a _non-blocking_ operation (like when multiple spiders work together), this is even better. 

## Create a project

Before you start scraping, you will have to set up a new Scrapy project. Enter a directory where you’d like to store your code and run:

```shell
$ scrapy startproject schools
```

This will create a `schools` directory with the following contents:

```shell
schools/
    scrapy.cfg            # deploy configuration file
    tutorial/             # project's Python module, you'll import your code from here
        __init__.py
        items.py          # project items definition file
        middlewares.py    # project middlewares file
        pipelines.py      # project pipelines file
        settings.py       # project settings file
        spiders/          # a directory where you'll later put your spiders
            __init__.py
```

## A simple (narrow) spider <a id='simple'> </a>

Spiders are classes that you define and that Scrapy uses to scrape information from a website (or a group of websites). They must subclass `scrapy.Spider` and define the initial requests to make, optionally how to follow links in the pages, and how to parse the downloaded page content to extract data.

This is the code for our first Spider. Save it in a file named `simple_spider.py` under the `schools/spiders` directory in your project:

```python
import scrapy

class SimpleSpider(scrapy.Spider):
    name = "simple"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
```

As you can see, our Spider subclasses `scrapy.Spider` and defines some attributes and methods:

-`name`: identifies the Spider. It must be unique within a project, that is, you can’t set the same name for different Spiders.

-`start_urls`: a list of URLs to provide the initial requests for the crawler. Armed with this list alone, the spider will download HTML from the webpages specified, much as a web browser does. But it won't extract anything from the pages--that's why we need to define the `parse()` method.

-`parse()`: a method that will be called to handle the response downloaded for each of the requests made. The response parameter is an instance of `TextResponse` that holds the page content and has further helpful methods to handle it.

The `parse()` method usually parses the response, extracting the scraped data as dicts and also finding new URLs to follow and creating new requests (`Request`) from them.

### How to run the spider

To put our spider to work, go to the project’s top level directory and run:

```shell
$ scrapy crawl simple
```

This command runs the spider with name `simple` that we’ve just added, that will send some requests for the `quotes.toscrape.com` domain. You will get an output similar to this:

```shell
...
2019-03-19 15:58:49 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2019-03-19 15:58:49 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2019-03-19 15:58:49 [scrapy.core.engine] INFO: Spider opened
2019-03-19 15:58:49 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2019-03-19 15:58:49 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://quotes.toscrape.com/robots.txt> (referer: None)
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/1/> (referer: None)
2019-03-19 15:58:49 [simple] DEBUG: Saved file quotes-1.html
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/2/> (referer: None)
2019-03-19 15:58:50 [simple] DEBUG: Saved file quotes-2.html
2019-03-19 15:58:50 [scrapy.core.engine] INFO: Closing spider (finished)
2019-03-19 15:58:50 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 678,
 'downloader/request_count': 3,
 ...}
2019-03-19 15:58:50 [scrapy.core.engine] INFO: Spider closed (finished)
```

Now, check the files in the current directory. You should notice that two new files have been created: `quotes-1.html` and `quotes-2.html`, with the content for the respective URLs, as our `parse` method instructs.

How did this work? Scrapy schedules the `scrapy.Request` objects returned by the `start_requests` method of the Spider. Upon receiving a response for each one, it instantiates `Response` objects and calls the callback method associated with the request (in this case, the `parse` method) passing the response as argument.

### Challenge
Modify and run the spider script above to scrape this short list of `start_urls`: 
```python
['http://www.baylessk12.org/', 'https://crcc.doniphanr1.k12.mo.us/', 'https://www.hazelwoodschools.org/southeastmiddle']
 ```

## Extracting data
The best way to learn how to extract data with Scrapy is trying selectors using the shell [Scrapy shell](https://docs.scrapy.org/en/latest/topics/shell.html#topics-shell). Remember to always enclose URLs in quotes (double quotes for Windows) when running Scrapy shell from command-line, otherwise urls containing arguments (ie. `&` character) will not work. Run:

```shell
$ scrapy shell 'http://quotes.toscrape.com'
```
You will see something like:

```shell
2019-03-19 20:00:05 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: tutorial)
2019-03-19 20:00:05 [scrapy.utils.log] INFO: Versions: lxml 4.3.1.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.15.0-45-generic-x86_64-with-Ubuntu-18.04-bionic
2019-03-19 20:00:05 [scrapy.crawler] INFO: Overridden settings: {...}
2019-03-19 20:00:05 [scrapy.extensions.telnet] INFO: Telnet Password: 030319d194e7f6b0
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
...]
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
...]
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2019-03-19 20:00:05 [scrapy.core.engine] INFO: Spider opened
2019-03-19 20:00:05 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://quotes.toscrape.com/robots.txt> (referer: None)
2019-03-19 20:00:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com> (referer: None)
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x7f07993c02b0>
... 
```

Using the shell, you can try selecting elements using CSS with the response object:
```shell
>>> response.css('title')
```
The result of running `response.css('title')` is a list-like object called SelectorList, which represents a list of Selector objects that wrap around XML/HTML elements and allow you to run further queries to fine-grain the selection or extract the data.

To extract the text from the title above, you can do:
```shell
>>> response.css('title::text').getall()
```

There are two things to note here: one is that we’ve added `::text` to the CSS query, to mean we want to select only the text elements directly inside `<title>` element. If we don’t specify `::text`, we’d get the full title element, including its tags:
```shell
>>> response.css('title').getall()
```
The other thing is that the result of calling `.getall()` is a list: it is possible that a selector returns more than one result, so we extract them all. When you know you just want the first result, as in this case, you can do:
```shell
>>> response.css('title::text').get()
```

Besides the `getall()` and `get()` methods, you can also use the `re()` method to extract using regular expressions:
```shell
>>> response.css('title::text').re(r'Quotes.*')
>>> response.css('title::text').re(r'Q\w+')
>>> response.css('title::text').re(r'(\w+) to (\w+)')
```
In order to find the proper CSS selectors to use, you can use your browser developer tools (e.g., in Chrome, right click > `Inspect`) to inspect the HTML and come up with a selector (for more info, see [Using your browser’s Developer Tools for scraping](https://docs.scrapy.org/en/latest/topics/developer-tools.html)). You can also try opening the response page from the shell in your web browser using `view(response)`.

### Challenge
Inspect [quotes.toscrape.com](quotes.toscrape.com) for the selectors associated with quotes. Use this information to display the text of one of the quotes in the scrapy shell. <br>
**Hint 1:** If you need help getting a better sense of website structure, use the HTML tree below (under "Extracting quotes and authors") as a visual guide.<br>
**Hint 2:** You can subset within selectors by using periods and spaces. For instance, the following produces a SelectorList for the class2 of each type2 within the class1 of each type1:
```shell
response.css('type1.class1 type2.class2')
```

Your solution here: 

```shell

```

### Extracting quotes and authors
Now that you know a bit about selection and extraction, let’s complete our spider by writing the code to extract the quotes from the web page.

Each quote in http://quotes.toscrape.com is represented by HTML elements that look like this:

```html
<div class="quote">
    <span class="text">“The world as we have created it is a process of our
    thinking. It cannot be changed without changing our thinking.”</span>
    <span>
        by <small class="author">Albert Einstein</small>
        <a href="/author/Albert-Einstein">(about)</a>
    </span>
    <div class="tags">
        Tags:
        <a class="tag" href="/tag/change/page/1/">change</a>
        <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
        <a class="tag" href="/tag/thinking/page/1/">thinking</a>
        <a class="tag" href="/tag/world/page/1/">world</a>
    </div>
</div>
```

How do we extract the data we want? To start, we get a list of selectors for the quote HTML elements with:

```shell
>>> response.css("div.quote")
```

Each of the selectors returned by the query above allows us to run further queries over their sub-elements. Let’s assign the first selector to a variable, so that we can run our CSS selectors directly on a particular quote:
```shell
>>> quote = response.css("div.quote")[0]
```

Now, let’s extract title, author and the tags from that quote using the quote object we just created:

```shell
>>> title = quote.css("span.text::text").get()
>>> title
>>> author = quote.css("small.author::text").get()
>>> author
```

Given that the tags are a list of strings, we can use the .getall() method to get all of them:
```shell
>>> tags = quote.css("div.tags a.tag::text").getall()
>>> tags
```

Having figured out how to extract each bit, we can now iterate over all the quotes elements and put them together into a Python dictionary. Copy and paste each of these subsequent lines into scrapy shell (or type them in via split-screen):
```shell
>>> for quote in response.css("div.quote"):
...     text = quote.css("span.text::text").get()
...     author = quote.css("small.author::text").get()
...     tags = quote.css("div.tags a.tag::text").getall()
...     print(dict(text=text, author=author, tags=tags))
>>>
```

### Extracting data in our spider
Let’s get back to our spider. Until now, it doesn’t extract any data in particular, just saves the whole HTML page to a local file. Let’s integrate the extraction logic above into our spider.

A Scrapy spider typically generates many dictionaries containing the data extracted from the page. To do that, we use the `yield` Python keyword in the callback, as you can see below:

```python
import scrapy


class SimpleSpider(scrapy.Spider):
    name = "simple"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }
```

If you run this spider, it will output the extracted data with the log:
```shell
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['life', 'love'], 'author': 'André Gide', 'text': '“It is better to be hated for what you are than to be loved for what you are not.”'}
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['edison', 'failure', 'inspirational', 'paraphrased'], 'author': 'Thomas A. Edison', 'text': "“I have not failed. I've just found 10,000 ways that won't work.”"}
```

### Storing the scraped data
The simplest way to store the scraped data is by using Feed exports, with the following command:
```shell
$ scrapy crawl quotes -o quotes.json
```

That will generate an quotes.json file containing all scraped items, serialized in JSON.

For historic reasons, Scrapy appends to a given file instead of overwriting its contents. If you run this command twice without removing the file before the second time, you’ll end up with a long JSON file--actually, a broken JSON file, which cannot be read.

You can also use other formats, like JSON Lines:
```shell
$ scrapy crawl quotes -o quotes.jl
```

The JSON Lines format is useful because it’s stream-like, you can easily append new records to it. It doesn’t have the same problem of JSON when you run twice. Also, as each record is a separate line, you can process big files without having to fit everything in memory.

In small projects (like the one in this tutorial), that should be enough. However, if you want to perform more complex things with the scraped items, you can write an Item Pipeline. A placeholder file for Item Pipelines has been set up for you when the project is created, in `schools/pipelines.py`. Though you don’t need to implement any item pipelines if you just want to store the scraped items.

## Link extraction in a (broad) spider <a id='linkextraction'> </a>

Let’s say, instead of just scraping the stuff from the first two pages from http://quotes.toscrape.com, you want quotes from all the pages in the website. We could do this by adding more URLs to the `start_urls` field, but this sounds like work. We could also read them in via a text file, but that doesn't sound any easier.

The best way would be to find the URLs of within-domain links from the website itself, extract data from these pages, go one level still further down, and so on. We could do this by identifying and extracting links using their CSS selectors, and this would let us scrape all the quotes across all the pages across this website (there are steps for this in [the `Scrapy` tutorial](https://docs.scrapy.org/en/latest/intro/tutorial.html)). But CSS selectors are site-specific, so what about other websites? Could we use `Scrapy` to crawl all the links reliably across different kinds of websites?

Yes! In fact, this kind of _broad crawling_ is (arguably) where `Scrapy` shines the most. Crawlers typically mix _horizontal crawling_, crawling pages at the same hierarchical level (for example, from `school.com/page1` to `school.com/page2`), with _vertical crawling_, moving from a higher hierarchical level (for example, `school.com/main`) to a lower one (for example, `school.com/main/about_us`). `Scrapy` makes doing both of these easy by providing the `CrawlSpider` class, from which we can borrow using the `genspider` command:

```shell
$ scrapy genspider -t crawl broad http://quotes.toscrape.com  # borrows from `crawl` spider template
Created spider 'broad' using template 'crawl' in module:
  schools.spiders.broad
```
Go ahead and execute this command in your terminal, and check out the resulting file in `schools/spiders/broad.py`. It should look like this:

```python
class BroadSpider(CrawlSpider):
    name = 'broad'
    allowed_domains = ['http://quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']

    rules = (
        Rule(LinkExtractor(allow=r'Items/'), callback='parse_item', follow=True),
    )

    def parse_item(self, response):
        item = {}
        ...
        return item
```
It's worth explaining this a bit. `CrawlSpider` provides an implementation of the `parse()` method that uses the `rules` variable to allow easy two-direction (both horizontal and vertical) crawling. For instance, `Scrapy` by default avoids duplicate requests. 

Unless a callback is set, a Rule will follow the extracted URLs, which means that it will scan target pages for extra links and follow them. If a callback is set, the Rule won't follow the links from target pages. If you would like it to follow links, you should either return/yield them from your callback method, or set the follow argument of Rule() to true (which is what we will do). 

How does this spider find links to follow? As their name implies, LinkExtractors are specialized in extracting links, so by default, they are looking for the `a` (and `area`) `href` HTML tags or attributes. Links are ordered using by  "last in, first out", meaning that it scrapes a page and all its sublinks before visiting the next page.

In a moment, we will adapt this template to scrape from all the pages on our sample website. But first, we need to update our `items.py` file.

### Defining items

Within the project directory, there’s an `items.py` file. Items add structure to our scraping results and are used by spiders.

Here you can add class fields such as url, images, or locations. These fields can be filled by pipelines (a more advanced topic).

Add to this file the fields for text, author, and tags. We will use these for our quotes spider.

```python
from scrapy.item import Item, Field

class SchoolsItem(Item):
    text = Field()
    author = Field()
    tags = Field()
```

Item properties can then be set with the response we get from parsing. 

```python
from schools.items import SchoolsItem
...
def parse_item(self, response):
    item = SchoolsItem()      
    item['text'] = ...
    item['author'] = ...
    item['tags'] = ...
    
    yield item
```

### Challenge

Adapt the `CrawlSpider` in `broad.py` to scrape the text, author, and tag for each quote across all the page on `http://quotes.toscrape.com`. Assign the `text`, `author`, and `tags` fields to Items, then yield the Items. Edit the spider script first, then run it via your terminal, then check the output to make sure.

```python
# Your solution here
```


### Challenge

Use what you learned above about removing tags with an exclusion list to rewrite the `parse_item()` function in the `BroadSpider()` so that it doesn't depend on website structure (HTML, CSS, XPath, etc.). In other words, write a truly broad crawler that only returns text.

Make sure to clean up the spacing: convert multiple newlines into a single one or a space, depending on the output format you want. 

_Hint:_ Check your output--is it missing anything important? Consider removing specific tags from the exclusion list. 


```python
# Your solution here
```